<a href="https://colab.research.google.com/github/prathyusha1311/Abstractive-Text-summarization-on-Movie-Reviews-using-PEGASUS/blob/main/Abstractive_text_summarization_on_Movie_Reviews_using_PEGASUS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Abstractive text summaeization using Pegasus - (Pre-training with Extracted Gap-sentences for Abstractive Summarization)

### Importing the required libraries 

In [ ]:
import pandas as pd
import re

### Installing Pytorch in the enviroment 

In [ ]:
# Install PyTorch
!pip install torch==1.8.2+cu111 torchvision==0.9.2+cu111 torchaudio===0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html


### Installing the transformer module along with their dependencies from Hugging Face 

Here the PEGASUS pre-trained transformer model has been used to solve the issue of Abstractive text summarization. A special XSum pegasus model has been utilized to perform summarization which generates a one line summary of the entire text which is inputted.

- Link to the pegasus research Paper : https://arxiv.org/pdf/1912.08777.pdf 
- Link to access the Google XSum Pegasus model from Hugging Face: https://huggingface.co/google/pegasus-xsum 




In [ ]:

!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Importing dependencies from transformers
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

### Dataset used: 

Dataset - https://ieee-dataport.org/open-access/imdb-movie-reviews-dataset 

Used IMDB dataset. This dataset contains nearly 1 Million unique movie reviews from 1150 different IMDb movies spread across 17 IMDb genres - Action, Adventure, Animation, Biography, Comedy, Crime, Drama, Fantasy, History, Horror, Music, Mystery, Romance, Sci-Fi, Sport, Thriller and War. The dataset also contains movie metadata such as date of release of the movie, run length, IMDb rating, movie rating (PG-13, R, etc), number of IMDb raters, and number of reviews per movie.

If The user choose to watch M0 ( A Beautiful Mind here), a summary is generated by the model in such way that it considers the history of movies the user has watched. Based on the History and the movie he has chosen a customized summary is generated specially for the user.

Even though the dataset is huge, only certain part of the dataset has only been utilized. 

Here in order to develop the abstractive text summariser only 10 movies are considered as watched by the user in the History.

#### **Data Preprocessing** 
1. In order to obtain all the review of the movie M0(A Beautiful Mind), the dataframe has been cleaned by dropping out unneccessary columns like - ['username','rating','helpful','total','date','title'] . 
2. Converting all the reviews (of string type) into a single text for easy and efficient working of the model and the special characters like $,@,? are also removed .



A beautiful mind review dataset summarization


In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/IAS/text summarization/Movies reviews/A Beautiful Mind 2001.csv")

In [ ]:
dataset

,username,rating,helpful,total,date,title,review
0,dfranzen70,9,129,152,23 January 2002,"Powerful, quiet, effortless Crowe\n",A Beautiful Mind<br/><br/>\nDirector Ron Howar...
1,The Mad Reviewer,10,250,312,16 January 2002,"A beautifully written, acted, and crafted mov...",A BEAUTIFUL MIND (2001) Rating: 10/10<br/><br/...
2,Boyo-2,10,212,269,17 December 2001,A Beautiful Movie\n,I think its a good idea to know as little as p...
3,jonwoolfrey,10,289,379,2 May 2002,A movie with a difference - It's realistic.\n,I went along to the movies not really wanting ...
4,susan_may21,9,175,230,5 September 2002,Direction makes it beautiful\n,"""A Beautiful Mind"" is an exceptional story, bu..."
...,...,...,...,...,...,...,...
1263,Unity019,10,0,1,2 January 2002,Finally a film that shows the humanity behind...,I felt this film was a breakthrough for Hollyw...
1264,craig_hutto,9,0,1,2 January 2002,Powerful\n,This film has great acting and a riveting stor...
1265,joaqoholic,Null,0,1,2 January 2002,"Beautiful Performances, Not Quite as Beautifu...",MINOR SPOILERS<br/><br/>Being an aspiring scie...
1266,reniperi,9,0,1,18 December 2001,A Beautiful Movie\n,"Strikingly rich, emotional, and turbulent, thi..."


In [ ]:

dataset.drop(['username','rating','helpful','total','date','title'], axis=1,inplace=True)

In [ ]:
dataset

,review
0,A Beautiful Mind<br/><br/>\nDirector Ron Howar...
1,A BEAUTIFUL MIND (2001) Rating: 10/10<br/><br/...
2,I think its a good idea to know as little as p...
3,I went along to the movies not really wanting ...
4,"""A Beautiful Mind"" is an exceptional story, bu..."
...,...
1263,I felt this film was a breakthrough for Hollyw...
1264,This film has great acting and a riveting stor...
1265,MINOR SPOILERS<br/><br/>Being an aspiring scie...
1266,"Strikingly rich, emotional, and turbulent, thi..."


In [ ]:

reviewM11 = dataset['review'].str.cat(sep=', ')


In [ ]:
preprocessed_text = reviewM11.strip().replace('<br/>','')

In [ ]:
#print(reviewM11)
reviewM11 = preprocessed_text

In [ ]:
reviewM11=re.sub("[$@&?]","",reviewM11)
print (reviewM11)

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Loading Tokenizers 

The tokenizer tokenizes each and every word in the text into tokens. Pre-trained transformer Pegasus XSum model has been used as a tokenizer.

In [ ]:
# Load tokenizer 
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

### Loading Model 

Similar to the tokenizer Pegasus XSum model has been used to create an abstract summary 

In [ ]:
# Load model 
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

### Creating tokens for the movie M0 - A Beautiful Mind movie review

In [ ]:
# Create tokens - number representation of our text
tokens = tokenizer(reviewM11, padding="longest", return_tensors="pt")

In [ ]:
print(tokens)

{'input_ids': tensor([[  202,  5290,  8450,  1970,  7836,  7720,   148,   306,   115,  1123,
           122,   169,  1684,   131,   116, 90484,   107,  4488,   115, 16198,
          7487,   173,   109,  9157,   113,   109, 29248,   140, 11899,   143,
         13086,   108,   167,   175,   119,  1173,   128,   909,   689,   108,
           119,  1606,  2810,   155,   309,  4038,  1453,  1173,   115, 31913,
           108,   173,  3948,  3374,   131,   116,  3721,   140,   160,   112,
           193,   109,  3236,  2602, 16749,  1141, 19670,   137,   309, 49710,
           274,  9127,   122,   109,   229,   113,   183,   107,   143,   996,
           119,   235,   108,   178,   131,   267,   394,   923,   270,   568,
          1141, 19670,   108,   254,   141,   274,   113,   214,   170,   394,
          1148,   139,  7441, 28838,  2945,   333,   203,  2061,   550,   107,
           158,   325, 73284, 90484,   117,   146,   114,  1025,   474,   134,
           149,   108,   146,   173,  

In [ ]:
# Summarize 
summary = model.generate(**tokens)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 64 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [ ]:
# Output summary tokens
summary[0]

tensor([    0,  8709, 44984,   111,  8019, 69383,   115,   114,  2166,   135,
          202,  5290,  8450,   107,     1])

In [ ]:
# Decode summary
tokenizer.decode(summary[0],skip_special_tokens=True)

'Russell Crowe and Jennifer Connelly in a scene from A Beautiful Mind.'

Made changes in the hyperparameters max_length (Maximum length of the one line summary) and min_length (Minimum words to be present in a one line summary) to create a summary of 50 words 

In [ ]:
summz = model.generate(**tokens, max_length= 150, min_length=20)

In [ ]:
summz[0]

tensor([    0,  8709, 44984,   111,  8019, 69383,  2187,   115,   202,  5290,
         8450,   108,   162,  3387,   109,   921,   584,   113, 50068,  1084,
        21006,   108,   170,   140,  6878,   122, 28603,   107,     1])

In [ ]:
review0 = tokenizer.decode(summz[0], skip_special_tokens=True)

In [ ]:
review0

'Russell Crowe and Jennifer Connelly star in A Beautiful Mind, which tells the true story of mathematician John Nash, who was diagnosed with schizophrenia.'

Considering 10 more movies as a history of the user :
the history contains (H1) ={ M1,M2,M3... M10} where M11 is A Beautiful Mind Movie

Comparing the movies in the History H1 and movie M11 review to give a customized review of the movie M11 (A Beautiful Mind) here. 

- M1 : The dark knight rises (Action)
- M2: Avengers - Endgame (Adventure)
- M3: Inside Out (Animation) 
- M4: The wolf of the wallstreet (biography)
- M5: The mask (Comedy)
- M6: Ocean's Eleven (Crime)
- M7: Titanic (Drama)
- M8: Beauty and the beast (Fantasy) 
- M9: Macbeth (History)
- M10: It (Horror)

the customized review comparing all the above movies and M11 (A Beautiful Mind ) 

A similar method proposed above has been used to create summaries of the reviews of the above mentioned 10 movies of different genres. 

### Movie1 - summary of the review

In [ ]:
#M1 - movie 1 

review_Movie_1 = pd.read_csv("/content/drive/MyDrive/IAS/text summarization/Movies reviews/The Dark Knight Rises 2012.csv")

In [ ]:
review_Movie_1

,username,rating,helpful,total,date,title,review
0,MR_Heraclius,10,54,57,12 February 2020,Batman\n,Christopher Nolan's epic trilogy concludes in ...
1,Geeky Randy,10,141,172,1 January 2013,A wonderful conclusion to a wonderful trilogy.\n,"After eight years in seclusion, Batman resurfa..."
2,darkslash7,10,58,69,20 July 2012,No one wraps up a trilogy like Nolan.\n,Wow. That's all I can say. I was freaked out a...
3,CaptKeshav,9,91,113,2 June 2013,Epic\n,The Dark Knight Rises was a perfect sequel to ...
4,bowwowwowyippy,9,14,15,13 November 2019,Astounding\n,Bane is a more terrifying nemesis than the Jok...
...,...,...,...,...,...,...,...
2948,kastri_gr,10,0,3,18 November 2012,Amazing but not quite so\n,The Dark Knight Rises film i am sure it would ...
2949,kumar_50,8,0,4,30 October 2012,Amazing Movie first time around...Not so much...,The 1st time i saw the movie i was so overwhel...
2950,phd_travel,7,0,6,28 October 2012,A bit confusing but quite grand and sweeping\n,There are some good points and some weaker poi...
2951,sunnysurfer101,8,0,3,24 October 2012,"I love it, but it's not as good as everyone s...","Okay, For starters, I LOVE THIS FILM. I REALLY..."


In [ ]:
review_Movie_1.drop(['username','rating','helpful','total','date','title'], axis=1,inplace=True)

In [ ]:
review_Movie_1


,review
0,Christopher Nolan's epic trilogy concludes in ...
1,"After eight years in seclusion, Batman resurfa..."
2,Wow. That's all I can say. I was freaked out a...
3,The Dark Knight Rises was a perfect sequel to ...
4,Bane is a more terrifying nemesis than the Jok...
...,...
2948,The Dark Knight Rises film i am sure it would ...
2949,The 1st time i saw the movie i was so overwhel...
2950,There are some good points and some weaker poi...
2951,"Okay, For starters, I LOVE THIS FILM. I REALLY..."


In [ ]:
reviewM1 = review_Movie_1['review'].str.cat(sep=', ')
reviewM1 = reviewM1.strip().replace('<br/>','')

In [ ]:
#reviewM1
#re.Replace(reviewM1, @"[^0-9a-zA-Z:,]+", "")  
reviewM1=re.sub("[$@&?]","",reviewM1)
#print (reviewM1)

In [ ]:
#tokenizer1 = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
# Load model 
#model1 = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
# Create tokens - number representation of our text
tokens1 = tokenizer(reviewM1, truncation=True, padding="longest", return_tensors="pt")
#print(tokens1)
summary1 = model.generate(**tokens1, max_length= 150, min_length=20)
#summary1[0]
review1 = tokenizer.decode(summary1[0],skip_special_tokens=True)
print(review1)

The third and final entry in Christopher Nolan's epic Dark Knight trilogy is a thrilling conclusion to an epic saga.


In [ ]:
review1

"The third and final entry in Christopher Nolan's epic Dark Knight trilogy is a thrilling conclusion to one of the greatest comic book sagas of all time, as Batman faces off against Bane, a man with ties to Bruce Wayne's past."

### movie 2 - summary of movie review  

In [ ]:
#M2 
review_Movie_2 = pd.read_csv("/content/drive/MyDrive/IAS/text summarization/Movies reviews/Avengers_ Endgame 2019.csv")
review_Movie_2.drop(['username','rating','helpful','total','date','title'], axis=1,inplace=True)
reviewM2 = review_Movie_2['review'].str.cat(sep=', ')
reviewM2 = reviewM2.strip().replace('<br/>','')
reviewM2=re.sub("[$@&?]","",reviewM2)
tokens2 = tokenizer(reviewM2, truncation=True, padding="longest", return_tensors="pt")
#print(tokens2)
summary2 = model.generate(**tokens2,max_length = 150, min_length=50)
review2 = tokenizer.decode(summary2[0],skip_special_tokens=True)

In [ ]:
review2

"After seeing Avengers: Infinity War for the first time, I have to say that it's the best movie I've ever seen in my whole life and I can't wait to see what the next one is going to be like."

### movie 3 - summary of movie review  

In [ ]:

#M3
review_Movie_3 = pd.read_csv("/content/drive/MyDrive/IAS/text summarization/Movies reviews/Inside Out 2015.csv")
review_Movie_3.drop(['username','rating','helpful','total','date','title'], axis=1,inplace=True)
reviewM3 = review_Movie_3['review'].str.cat(sep=', ')
reviewM3 = reviewM3.strip().replace('<br/>','')
reviewM3=re.sub("[$@&?]","",reviewM3)
tokens3 = tokenizer(reviewM3, truncation=True, padding="longest", return_tensors="pt")
#print(tokens3)
summary3 = model.generate(**tokens3, max_length= 150, min_length=50)
review3 = tokenizer.decode(summary3[0],skip_special_tokens=True)

In [ ]:
review3

"Riley's Story is one of the best Pixar films I have ever seen, and I have to say, I am a huge fan of it, even though I have heard a lot of people say that it is too intense for young children."

### movie 4 - summary of movie review  

In [ ]:
#M4
review_Movie_4 = pd.read_csv("/content/drive/MyDrive/IAS/text summarization/Movies reviews/The Wolf of Wall Street 2013.csv")
review_Movie_4.drop(['username','rating','helpful','total','date','title'], axis=1,inplace=True)
reviewM4 = review_Movie_4['review'].str.cat(sep=', ')
reviewM4 = reviewM4.strip().replace('<br/>','')
reviewM4=re.sub("[$@&?]","",reviewM4)
tokens4 = tokenizer(reviewM4, truncation=True, padding="longest", return_tensors="pt")
#print(tokens4)
summary4 = model.generate(**tokens4, max_length= 150, min_length=50)
review4 = tokenizer.decode(summary4[0],skip_special_tokens=True)

In [ ]:
review4

"Martin Scorcese's The Wolf of Wall Street tells the story of disgraced Wall Street financier Jordan Belfort (Leonardo DiCaprio) and the debauchery that goes along with his rise and fall from the top of the stock market to the bottom."

### movie 5 - summary of movie review  

In [ ]:
#M5 
review_Movie_5 = pd.read_csv("/content/drive/MyDrive/IAS/text summarization/Movies reviews/The Mask 1994.csv")
review_Movie_5.drop(['username','rating','helpful','total','date','title'], axis=1,inplace=True)
reviewM5 = review_Movie_5['review'].str.cat(sep=', ')
reviewM5 = reviewM5.strip().replace('<br/>','')
reviewM5=re.sub("[$@&?]","",reviewM5)
tokens5 = tokenizer(reviewM5, truncation=True, padding="longest", return_tensors="pt")
#print(tokens5)
summary5 = model.generate(**tokens5, max_length= 150, min_length=50)
review5 = tokenizer.decode(summary5[0],skip_special_tokens=True)

In [ ]:
review5

'What do you think of Jim Carrey\'s performance in "The Mask", one of his best acting roles of all time, and what do you think of the film as a whole, and what do you think of Jim Carrey\'s performance in it?'

### movie 6 - summary of movie review  

In [ ]:
#M6
review_Movie_6 = pd.read_csv("/content/drive/MyDrive/IAS/text summarization/Movies reviews/Ocean's Eleven 2001.csv")
review_Movie_6.drop(['username','rating','helpful','total','date','title'], axis=1,inplace=True)
reviewM6 = review_Movie_6['review'].str.cat(sep=', ')
reviewM6 = reviewM6.strip().replace('<br/>','')
reviewM6=re.sub("[$@&?]","",reviewM6)
tokens6 = tokenizer(reviewM6, truncation=True, padding="longest", return_tensors="pt")
#print(tokens6)
summary6 = model.generate(**tokens6, max_length= 150, min_length=50)
review6 = tokenizer.decode(summary6[0],skip_special_tokens=True)


In [ ]:
review6

"Ocean's Eight is one of the best movies I've seen in a long time, and one of the best movies I've seen in a long time because it's so slick, it's so entertaining and it's so well made."

### movie 7 - summary of movie review  

In [ ]:
#M7
review_Movie_7 = pd.read_csv("/content/drive/MyDrive/IAS/text summarization/Movies reviews/Titanic 1997.csv")
review_Movie_7.drop(['username','rating','helpful','total','date','title'], axis=1,inplace=True)
reviewM7 = review_Movie_7['review'].str.cat(sep=', ')
reviewM7 = reviewM7.strip().replace('<br/>','')
reviewM7=re.sub("[$@&?]","",reviewM7)
tokens7 = tokenizer(reviewM7, truncation=True, padding="longest", return_tensors="pt")
#print(tokens7)
summary7 = model.generate(**tokens7, max_length= 150, min_length=50)
review7 = tokenizer.decode(summary7[0],skip_special_tokens=True)

In [ ]:
review7

"With the 20th anniversary of the sinking of the Titanic just around the corner, we asked some of you to tell us what you think about the film - and here's what you had to say: Titanic is one of my all time favourite films."

### movie 8 - summary of movie review  

In [ ]:

#M8
review_Movie_8 = pd.read_csv("/content/drive/MyDrive/IAS/text summarization/Movies reviews/Beauty and the Beast 2017.csv")
review_Movie_8.drop(['username','rating','helpful','total','date','title'], axis=1,inplace=True)
reviewM8 = review_Movie_8['review'].str.cat(sep=', ')
reviewM8 = reviewM8.strip().replace('<br/>','')
reviewM8=re.sub("[$@&?]","",reviewM8)
tokens8 = tokenizer(reviewM8, truncation=True, padding="longest", return_tensors="pt")
#print(tokens8)
summary8 = model.generate(**tokens8, max_length= 150, min_length=50)
review8 = tokenizer.decode(summary8[0],skip_special_tokens=True)
#review8 = review8.strip().replace('\'',"'")


In [ ]:
#review8.replace("\'", "'")
#"\'".replace("\'","'")

In [ ]:
review8

'Disney\'s live-action remake of Beauty and the Beast has received a 7.8 out of 10 on IMDb, with one critic calling it "disappointing" and another saying it\'s "pointless" and "a waste of time."'

### movie 9 - summary of movie review  

In [ ]:
#M9
review_Movie_9 = pd.read_csv("/content/drive/MyDrive/IAS/text summarization/Movies reviews/Macbeth 2015.csv")
review_Movie_9.drop(['username','rating','helpful','total','date','title'], axis=1,inplace=True)
reviewM9 = review_Movie_9['review'].str.cat(sep=', ')
reviewM9 = reviewM9.strip().replace('<br/>','')
reviewM9=re.sub("[$@&?]","",reviewM9)
tokens9 = tokenizer(reviewM9, truncation=True, padding="longest", return_tensors="pt")
#print(tokens9)
summary9 = model.generate(**tokens9, max_length= 150, min_length=50)
review9 = tokenizer.decode(summary9[0],skip_special_tokens=True)

In [ ]:
review9

'Michael Fassbender and Marion Cotillard as Macbeth and Lady Macbeth were absolutely mind- blowing and amazing and encapsulated these characters as perfectly as you could get. Michael Fassbender and Marion Cotillard as Macbeth and Lady Macbeth were absolutely mind- blowing and amazing and encapsulated these characters as perfectly as you could get.'

### movie 10 - summary of movie review  

In [ ]:
#M10
review_Movie_10 = pd.read_csv("/content/drive/MyDrive/IAS/text summarization/Movies reviews/It 2017.csv")
review_Movie_10.drop(['username','rating','helpful','total','date','title'], axis=1,inplace=True)
reviewM10 = review_Movie_10['review'].str.cat(sep=', ')
reviewM10 = reviewM10.strip().replace('<br/>','')
reviewM10=re.sub("[$@&?]","",reviewM10)
tokens10 = tokenizer(reviewM10, truncation=True, padding="longest", return_tensors="pt")
#print(tokens10)
summary10 = model.generate(**tokens10, max_length= 150, min_length=50)
review10 = tokenizer.decode(summary10[0],skip_special_tokens=True)

In [ ]:
review10

"Here's my review of IT, which is based on Stephen King's novel of the same name and tells the story of a group of children who go missing in a small town in the 1980s after a clown called Pennywise shows up."

Combining all the 11 reviews into a dictionary 'review'

In [ ]:
#review = list()
# review[0] contains the summary of the movie that user has picked to watch (M11)
#review[0][0] = review0
review = {0: review0, 1: review1, 2: review2, 3: review3, 4: review4, 5: review5, 6:review6, 7:review7, 8: review8, 9: review9, 10:review10 }

In [ ]:
print(review)

{0: 'Russell Crowe and Jennifer Connelly star in A Beautiful Mind, a film from Ron Howard, based on the true story of mathematician John Nash, who was diagnosed with a form of schizophrenia in the 1960s, and who spent time in a sanitarium.', 1: "The third and final entry in Christopher Nolan's epic Dark Knight trilogy is a thrilling conclusion to one of the greatest comic book sagas of all time, as Batman faces off against Bane, a man with ties to Bruce Wayne's past.", 2: "After seeing Avengers: Infinity War for the first time, I have to say that it's the best movie I've ever seen in my whole life and I can't wait to see what the next one is going to be like.", 3: "Riley's Story is one of the best Pixar films I have ever seen, and I have to say, I am a huge fan of it, even though I have heard a lot of people say that it is too intense for young children.", 4: "Martin Scorcese's The Wolf of Wall Street tells the story of disgraced Wall Street financier Jordan Belfort (Leonardo DiCaprio)

In [ ]:
summary_final = ''
for i in range (0,11):
  #str1 = str(i)
  summary_final = summary_final+ review[i]
print(summary_final)

Russell Crowe and Jennifer Connelly star in A Beautiful Mind, a film from Ron Howard, based on the true story of mathematician John Nash, who was diagnosed with a form of schizophrenia in the 1960s, and who spent time in a sanitarium.The third and final entry in Christopher Nolan's epic Dark Knight trilogy is a thrilling conclusion to one of the greatest comic book sagas of all time, as Batman faces off against Bane, a man with ties to Bruce Wayne's past.After seeing Avengers: Infinity War for the first time, I have to say that it's the best movie I've ever seen in my whole life and I can't wait to see what the next one is going to be like.Riley's Story is one of the best Pixar films I have ever seen, and I have to say, I am a huge fan of it, even though I have heard a lot of people say that it is too intense for young children.Martin Scorcese's The Wolf of Wall Street tells the story of disgraced Wall Street financier Jordan Belfort (Leonardo DiCaprio) and the debauchery that goes alo

### Final customized summary for the movie Beautiful Mind 
This final summary takes into the consideration of all the movies the user has watched from the history and generates a customized review/ summary the user picked for watching. 

In [ ]:
summary_final

'Russell Crowe and Jennifer Connelly star in A Beautiful Mind, a film from Ron Howard, based on the true story of mathematician John Nash, who was diagnosed with a form of schizophrenia in the 1960s, and who spent time in a sanitarium.The third and final entry in Christopher Nolan\'s epic Dark Knight trilogy is a thrilling conclusion to one of the greatest comic book sagas of all time, as Batman faces off against Bane, a man with ties to Bruce Wayne\'s past.After seeing Avengers: Infinity War for the first time, I have to say that it\'s the best movie I\'ve ever seen in my whole life and I can\'t wait to see what the next one is going to be like.Riley\'s Story is one of the best Pixar films I have ever seen, and I have to say, I am a huge fan of it, even though I have heard a lot of people say that it is too intense for young children.Martin Scorcese\'s The Wolf of Wall Street tells the story of disgraced Wall Street financier Jordan Belfort (Leonardo DiCaprio) and the debauchery that 

In [ ]:
import textwrap
  
#value = """This function wraps the input paragraph such that each line
#in the paragraph is at most width characters long. The wrap method
#returns a list of output lines. The returned list
#is empty if the wrapped
#output has no content."""
  
# Wrap this text.
wrapper = textwrap.TextWrapper(width=100)
  
word_list = wrapper.wrap(text=summary_final)
  
# Print each line.
for element in word_list:
    print(element)

Russell Crowe and Jennifer Connelly star in A Beautiful Mind, a film from Ron Howard, based on the
true story of mathematician John Nash, who was diagnosed with a form of schizophrenia in the 1960s,
and who spent time in a sanitarium.The third and final entry in Christopher Nolan's epic Dark Knight
trilogy is a thrilling conclusion to one of the greatest comic book sagas of all time, as Batman
faces off against Bane, a man with ties to Bruce Wayne's past.After seeing Avengers: Infinity War
for the first time, I have to say that it's the best movie I've ever seen in my whole life and I
can't wait to see what the next one is going to be like.Riley's Story is one of the best Pixar films
I have ever seen, and I have to say, I am a huge fan of it, even though I have heard a lot of people
say that it is too intense for young children.Martin Scorcese's The Wolf of Wall Street tells the
story of disgraced Wall Street financier Jordan Belfort (Leonardo DiCaprio) and the debauchery that
goes alo

### Challenges: 
- As we can see from the above result of the text summarization, few words are not understandable and in some cases the lines seem to be repeated again in order to get an appropriate output. 
- As the IMDB Dataset of these movies has been used, which contain reviews of the movie by people around the world, most of them expressed in first person like in 'I/We'. This makes the summarizer produce summary which contains the I / We pronouns which are not expected in the summary (Summary always are in 3rd person prespectives ). 
- The Pegasus model could create a one line summary only. When a 30 word containing one line summary is expected, the model doesn't give good results.


### Conclusion: 
The results obtained didn't match with the expectation. Major Challenge was to find the dataset to solve the above problem. With very limited data to work with there appears the problem of inefficiency and poor performance. 